In [40]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [41]:
#train_data = pd.read_csv('austin_weather.csv')['TempAvgF']
train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')['Sea_Surface_Temp']
train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:10000]
print(train_data)
train_data.shape

[-0.75397095 -0.89195707 -1.24458829 ... -0.0742615  -0.04870851
 -0.08448269]


(10000,)

In [42]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,32)
        self.lstm2 = nn.LSTM(32,32)
        self.lstm3 = nn.LSTM(32,1)
 
    def forward(self, seq):
        lstm1_out, _ = self.lstm1(seq, None)
        lstm2_out, _ = self.lstm2(lstm1_out, None)
        lstm3_out, _ = self.lstm3(lstm2_out, None)
        return lstm3_out.squeeze(1)

In [43]:
seq = Sequence()
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.008)

In [ ]:
for epoch in range(50):
    total_loss = 0
    passes = 0
    for i in range(0, 10000, 5):
        data = train_data[i:i+32]
        xs = torch.Tensor(data[:-1]).view(-1,1,1)
        ys = torch.Tensor(data[1:])
        
        lstm_out = seq(xs)
        loss = criterion(lstm_out.view(-1), ys)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data[0]
        passes += 1
        
    print("epoch {}, loss {:2.1%}".format(epoch, total_loss/passes))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


epoch 0, loss 148.8%
epoch 1, loss 142.4%
epoch 2, loss 144.2%
epoch 3, loss 142.1%
epoch 4, loss 141.9%
epoch 5, loss 142.1%
